## Decorator

Decorator is a function that takes another functions, and also extend the behaviour of these functions without explictly modifying it.

### Functions

Everything in Python is Object. Function is in first class, which means function can be passed around and used as arguments

In [ ]:
# passing function as arguments
def say_hello(name):
    return f'Hello {name}'

def be_awesome(name):
    return f'Hi, {name}. We are awesome!'

def greet(func):
    return func('Faris')

greet(say_hello)

In [ ]:
 # inner function
 def parent():
     def first_child():
         print("first child")
     def second_child():
         print("second child")

     print('parent')
     first_child()
     second_child()

parent()

In [ ]:
# return function from function
def parent(num):
    def first_child():
        print("first child")
    def second_child():
        print("second child")
    return first_child if num == 1 else second_child


#first is just a reference of function first_child
first = parent(1)
second = parent(2)

#take two braces to call this function.
first()
second()

### Simple Decorator

sometimes, we can implement simple docorator by using inner functions.

In [ ]:
# simple decorator
def simple_decorator(func):
    def wrapper():
        print("something happend before the function is called")
        func()
        print("something happend after the function is called")
    return wrapper

def say_hi():
    print('Hi')

# hi is a reference of wrapper function, which means it wrapper the say_hi function, and enhance say_hi behaviour.
hi = simple_decorator(say_hi)
print(hi, '\n')
hi()

In [ ]:
# Syntactic Sugar without arguments
def simple_decorator(func):
    def wrapper():
        print("do something before calling funcs")
        func()
    return wrapper

@simple_decorator
def say_hi():
    print('Hi')

say_hi()

In [ ]:
# with arguments, *args, **kargs means it can accept an arbitrary number of positional and keyword arguments.
def simple_decorator(func):
    def wrapper(*args, **kargs):
        print("do something before calling funcs")
        func(*args, **kargs)
    return wrapper

@simple_decorator
def greet(name):
    print(f'Hi, {name}')

greet('Faris')

In [ ]:
# return value from Decorated function
def simple_decorator(func):
    def wrapper(*args, **kargs):
        print("do something before calling funcs")
        return func(*args, **kargs)
    return wrapper

@simple_decorator
def greet(name):
    return f'Hi, {name}'

result = greet('Faris')
result

In [ ]:
# However, examples above lost their introspection ability, which means they do not know about their attributes at runtime. For instance, a function need to know its own name and documentation

print(greet, '\n')
print(greet.__name__, '\n')
help(greet)

We can see greet function has gotten confused about its identity. Now, it point to the wrapper method. 

In order to fix this issue, the decorator function should use `@functools.wraps`, which can preserve information about the original functions.

In [ ]:
import functools

def simple_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kargs):
        print("do something before calling funcs")
        return func(*args, **kargs)
    return wrapper

@simple_decorator
def greet(name):
    return f'Hi, {name}'

print(greet, '\n')
print(greet.__name__, '\n')
help(greet)

### Example

#### Timing Functions
It will measure a time which the function takes to execute and print the duration to console

In [ ]:
import functools
import time

def timer(func):
    @functools.wraps(func)
    def wrapper(*args, **kargs):
        start_time = time.perf_counter()
        value = func(*args, **kargs)
        end_time = time.perf_counter()
        interval = end_time - start_time
        print(f'Finished {func.__name__!r} in {interval:.4f} secs')
        return value
    return wrapper

@timer
def waste_time(num):
    print('sum:', sum(list(range(1, num + 1))))

waste_time(10)

#### Debug

It will print the arguments a function is called with as well as its return value every time the function is called.

In [ ]:
import functools

def debug(func):

    @functools.wraps(func)
    def wrapper(*args, **kargs):
        args_repr = [repr(item) for item in args]
        kwargs_repr = [f'{key} = {value!r}' for key, value in kargs.items()]
        signatures = ", ".join(args_repr + kwargs_repr)
        print(f'Calling {func.__name__}({signatures})')
        value = func(*args, **kargs)
        print(f'{func.__name__} returns {value!r}')
        return value
    return wrapper

@debug
def say_hi(name):
    return f'Hi, {name}'

say_hi('Faris')

### Nesting Decorator

it means we can apply several decorators to one function 

In [ ]:
#please notice the order of executing this say_hi. And compare to say_hi_1 

@debug
@timer
# the order of executing is like calling debug(timer(say_hi))
def say_hi(name):
    print(f'Hi, {name}')

say_hi('Faris')
print('\n\n')

@timer
@debug
# the order of executing is like calling timer(debug(say_hi))
def say_hi_1(name):
    print(f'How are you?, {name}')

say_hi_1('Faris')

### Decorator with arguments


In [ ]:
import functools

def repeat(_func=None, *, num_times=2):
    def repeat_decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kargs):
            for _ in range(num_times):
                value = func(*args, **kargs)
            return value
        return wrapper
    if _func is None:
        return repeat_decorator
    else:
        return repeat_decorator(_func)

@repeat
def say_hi():
    print('Hi')

@repeat(num_times=3)
def say_hello():
    print('Hello')

say_hi()
say_hello()

### Decorator that can keep tracking of state

As we know, functions are also objects in Python. functions will be created once when interpreter loads them.

In [ ]:
import functools

def count_call(func):
    @functools.wraps(func)
    def wrapper(*args, **kargs):
        wrapper.num_call += 1
        print(f'Calling {wrapper.num_call} of {func.__name__!r}')
        return func(*args, **kargs)
    wrapper.num_call = 0
    return wrapper

@count_call
def say_hi():
    print('Hi')

say_hi()
say_hi()
say_hi()

### Classes as Decorator

The typical way to maintain state is by using Class

As we know, the easier way to descirbe decorator is `func = decorator_class(fucs)`. which means we have to take func as an arguments into `__init__` functoin, and also make this class `callable` to implement `__call__`


Note we should use `functools.update_wrapper` instead of `@functools.wraps`

In [ ]:
import functools

class CountCalls:

    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_call = 0

    def __call__(self, *args, **kargs):
        self.num_call += 1
        print(f'Call {self.num_call} of {self.func.__name__!r}')
        return self.func(*args, **kargs)


@CountCalls
def say_hi(name):
    print(f'Hi, {name}')


say_hi('Faris')
say_hi('Faris1')

### Singleton

In [ ]:
import functools

def singleton(cls):
    @functools.wraps(cls)
    def wrapper(*args, **kargs):
        if not wrapper.instance:
            wrapper.instance = cls(*args, **kargs)
        return wrapper.instance

    wrapper.instance = 0
    return wrapper

@singleton
class TestSingleton:
    def __init__(self):
        pass

print(TestSingleton())
print(TestSingleton())